<a href="https://colab.research.google.com/github/njucs/notebook/blob/master/FirstTry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **准备工作**

import 导入模块，每次使用模块中的函数都要是定是哪个模块。 from … import * 导入模块，每次使用模块中的函数，直接使用函数就可以了；注因为已经知道该函数是那个模块中的了。

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import torchvision as tv
from torchvision import models,transforms,datasets

### **一些小技巧**

In [30]:
# 查看Python解释器
import sys
print(sys.executable)

# 测试GPU是否可用
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Using gpu: %s ' % torch.cuda.is_available())
use_gpu = torch.cuda.is_available()

# 把Tensor转成Image，方便可视化
'''
from torchvision.transforms import ToPILImage
show = ToPILImage()

x = torch.randn(300,500)
show(x)#.resize((100, 100))
'''

/usr/bin/python3
Using gpu: True 


### **数据加载和预处理**
**Dataset**对象是一个数据集，可以按下标访问，返回形如(data, label)的数据。

**Dataloader**是一个可迭代的对象，它将dataset返回的每一条数据拼接成一个batch，并提供多线程加速优化和数据打乱等操作。当程序对dataset的所有数据遍历完一遍之后，相应的对Dataloader也完成了一次迭代。

In [ ]:
import torchvision.transforms as transforms

# 定义对数据的预处理
transform = transforms.Compose([
        transforms.ToTensor(), # 转为Tensor
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), # 归一化
                             ])

# 训练集
trainset = tv.datasets.CIFAR10(
                    root='./data/tmp/', 
                    train=True, 
                    download=True,
                    transform=transform)

trainloader = torch.utils.data.DataLoader(
                    trainset, 
                    batch_size=4,
                    shuffle=True, 
                    num_workers=2)

# 测试集
testset = tv.datasets.CIFAR10(
                    './data/tmp/',
                    train=False, 
                    download=True, 
                    transform=transform)

testloader = torch.utils.data.DataLoader(
                    testset,
                    batch_size=4, 
                    shuffle=False,
                    num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# 可以查看一下部分数据内容
'''
dataiter = iter(trainloader)
images, labels = dataiter.next() # 返回4张图片及标签
print(' '.join('%11s'%classes[labels[j]] for j in range(4)))
show(tv.utils.make_grid((images + 1) / 2)).resize((400,100))
#show(images[2]).resize((100,100))
'''

### **定义网络**
定义网络时，需要继承nn.Module，并实现它的forward方法，**把网络中具有可学习参数的层放在构造函数\__init__中**。如果某一层(如ReLU)不具有可学习的参数，则既可以放在构造函数中，也可以不放，但建议不放在其中，而在forward中使用nn.functional代替。

**只要在nn.Module的子类中定义了forward函数，backward函数就会自动被实现(利用autograd)**。在forward 函数中可使用任何tensor支持的函数，还可以使用if、for循环、print、log等Python语法，写法和标准的Python写法一致。

torch.nn只支持mini-batches，不支持一次只输入一个样本，即一次必须是一个batch。但如果只想输入一个样本，则用 input.unsqueeze(0)将batch_size设为１。即输入必须是N个samples，但N可以设为1。

In [ ]:
class Net(nn.Module):
    def __init__(self):
        # nn.Module子类的函数必须在构造函数中执行父类的构造函数
        # 下式等价于nn.Module.__init__(self)
        super(Net, self).__init__()
        
        # 卷积层
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        # 全连接层
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # reshape，‘-1’表示自适应
        x = x.view(x.size()[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
print(net)

if(use_gpu):
    net = net.cuda()

### **查看网络的可学习参数**

网络的可学习参数通过net.parameters()返回，net.named_parameters可同时返回可学习的参数及名称。

In [ ]:
params = list(net.parameters())
print(params)

for name,parameters in net.named_parameters():
    print(name,':',parameters.size())

### **定义损失函数和优化器**

In [26]:
# 损失函数
criterion = nn.CrossEntropyLoss()
'''
criterion = nn.MSELoss() # 均方误差损失, 计算 output 和 target 之差的均方差.
criterion = nn.CrossEntropyLoss() # 交叉熵损失函数, 描述两个概率分布的差异, 当训练有 C 个类别的分类问题时很有效.
criterion = nn.KLDivLoss() # 计算 input 和 target 之间的 KL 散度. KL 散度可用于衡量不同的连续分布之间的距离, 在连续的输出分布的空间上(离散采样)上进行直接回归时很有效.
criterion = nn.BCELoss() # 二进制交叉熵损失 BCELoss. 二分类任务时的交叉熵计算函数. 注意目标的值的范围为0到1之间.
criterion = nn.MultiLabelMarginLoss() # 多标签分类损失 MultiLabelMarginLoss
criterion = nn.MultiLabelSoftMarginLoss() # 多标签 one-versus-all 损失
criterion = nn.CosineEmbeddingLoss() # cosine 损失
criterion = nn.MultiMarginLoss(p=1, margin=1.0) # 多类别分类的hinge损失
criterion = nn.TripletMarginLoss(margin=1.0, p=2.0, eps=1e-06, swap=False, reduction='mean') # 三元组损失
criterion = nn.NLLLoss() # 负对数似然损失. 用于训练 C 个类别的分类问题.
'''

# 优化器
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
'''
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adagrad(net.parameters(), lr=0.01, lr_decay=0, weight_decay=0, initial_accumulator_value=0) # 一种自适应优化方法，是自适应的为各个参数分配不同的学习率
optimizer = optim.RMSprop(net.parameters(), lr=0.01, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0, centered=False) # 对Adagrad的一种改进，可缓解Adagrad学习率下降较快的问题
optimizer = optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False) # 结合了Momentum和RMSprop，并进行了偏差修正
'''

if(use_gpu):
    criterion = criterion.cuda()

### **训练网络并更新网络参数**

所有网络的训练流程都是类似的，不断地执行如下流程：

1. 输入数据
2. 前向传播+反向传播
3. 更新参数

In [ ]:
torch.set_num_threads(8)
for epoch in range(20):  
    
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        
        # 输入数据
        inputs, labels = data
        if(use_gpu):
            inputs = inputs.cuda()
            labels = labels.cuda()
        
        # 梯度清零
        optimizer.zero_grad()
        
        # forward + backward 
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()   
        
        # 更新参数 
        optimizer.step()
        
        # 打印log信息
        # loss 是一个scalar,需要使用loss.item()来获取数值，不能使用loss[0]
        running_loss += loss.item()
        if i % 2000 == 1999: # 每2000个batch打印一下训练状态
            print('[%d, %5d] loss: %.3f' \
                  % (epoch+1, i+1, running_loss / 2000))
            running_loss = 0.0
print('Finished Training')

### **测试网络**
测试部分看看效果

In [ ]:
'''
dataiter = iter(testloader)
images, labels = dataiter.next() # 一个batch返回4张图片
images, labels = images.to(device), labels.to(device)

print('实际的label: ', ' '.join(\
            '%08s'%classes[labels[j]] for j in range(4)))
show(tv.utils.make_grid(images / 2 - 0.5)).resize((400,100))

# 计算图片在每个类别上的分数
outputs = net(images)
# 得分最高的那个类
_, predicted = torch.max(outputs.data, 1)

print('预测结果: ', ' '.join('%5s'\
            % classes[predicted[j]] for j in range(4)))
'''

完整的测试结果

In [ ]:
correct = 0 # 预测正确的图片数
total = 0 # 总共的图片数

# 由于测试的时候不需要求导，可以暂时关闭autograd，提高速度，节约内存
with torch.no_grad():
    for data in testloader:
        images, labels = data
        if(use_gpu):
            images = images.cuda()
        outputs = net(images)
        if(use_gpu):
            outputs = outputs.cpu()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

print('10000张测试集中的准确率为: %d %%' % (100 * correct / total))